# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import re
import tensorflow
import numpy as np
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Embedding
from sklearn.model_selection import train_test_split
import json
import requests
import time
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
import random

# Step 1: Data Preprocessing
def preprocess_data(dictionary_file, sample_size=None):
    # Load words from dictionary
    with open(dictionary_file, "r") as f:
        words = f.read().splitlines()

    # Optionally sample a subset of words
    if sample_size:
        words = random.sample(words, sample_size)

    # Create character mappings
    letters = string.ascii_lowercase
    char_to_idx = {ch: idx + 1 for idx, ch in enumerate(letters)}
    idx_to_char = {idx + 1: ch for idx, ch in enumerate(letters)}

    max_word_len = 20
    encoded_words = []

    # Encode and pad words
    for word in words:
        encoded_word = [char_to_idx.get(ch, 0) for ch in word[:max_word_len]]
        padded_word = [0] * (max_word_len - len(encoded_word)) + encoded_word
        encoded_words.append(padded_word)

    # Generate input sequences and labels
    X, y = [], []
    for word in encoded_words:
        for i in range(1, len(word)):
            X.append(word[:i])
            y.append(word[i])

    # Convert to numpy arrays and pad sequences
    X = np.array([np.pad(seq, (max_word_len - len(seq), 0), 'constant') for seq in X])
    y = np.array(y)

    return X, y, char_to_idx, idx_to_char

# Step 2: Building the LSTM Model
def build_lstm_model(vocab_size, input_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=64, input_length=input_length),
        Bidirectional(LSTM(128)),
        Dense(vocab_size, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Step 3: Training the Model
def train_model(X, y, vocab_size):
    input_length = X.shape[1]
    model = build_lstm_model(vocab_size, input_length)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train with batch size 64
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

    return model

# Main script to preprocess data and train the model
sample_size = None  # Set a sample size if needed, otherwise use the full dataset
X, y, char_to_idx, idx_to_char = preprocess_data("words_250000_train.txt", sample_size)

vocab_size = len(char_to_idx) + 1  # Adding 1 for padding
lstm_model = train_model(X, y, vocab_size)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 407s 7ms/step - accuracy: 0.6367 - loss: 1.3124 - val_accuracy: 0.6696 - val_loss: 1.1983
Epoch 2/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 446s 8ms/step - accuracy: 0.6721 - loss: 1.1893 - val_accuracy: 0.6756 - val_loss: 1.1802
Epoch 3/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 433s 7ms/step - accuracy: 0.6780 - loss: 1.1705 - val_accuracy: 0.6789 - val_loss: 1.1703
Epoch 4/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 486s 8ms/step - accuracy: 0.6817 - loss: 1.1593 - val_accuracy: 0.6806 - val_loss: 1.1658
Epoch 5/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 463s 7ms/step - accuracy: 0.6838 - loss: 1.1528 - val_accuracy: 0.6810 - val_loss: 1.1626
Epoch 6/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 444s 8ms/step - accuracy: 0.6847 - loss: 1.1501 - val_accuracy: 0.6817 - val_loss: 1.1606
Epoch 7/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 401s 7ms/step - accuracy: 0.6859 - loss: 1.1454 - val_accuracy: 0.6829 - val_loss: 1.1583
Epoch 8/10
53984/53984 ━━━━━━━━━━━━━━━━━━━━ 441s 7ms/step - ac

In [11]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        # Clean and prepare the word for the model (remove underscores, replace with padding indicator)
        clean_word = word.replace("_", "0")

        # Convert characters to indices using char_to_idx mapping
        input_seq = [char_to_idx.get(ch, 0) for ch in clean_word]

        # Ensure that the input sequence length does not exceed the maximum length (truncate if necessary)
        if len(input_seq) > 20:
            input_seq = input_seq[:20]

        # Pad the sequence to the expected input length for the model if it's shorter than the max length
        input_seq = np.pad(input_seq, (20 - len(input_seq), 0), mode='constant', constant_values=0)

        # Prepare input for prediction (batch size of 1)
        input_seq = np.array([input_seq])

        # Get model predictions
        predictions = lstm_model.predict(input_seq)
        pred_idx = np.argmax(predictions[0])

        # Map the predicted index back to a character
        guess_letter = idx_to_char.get(pred_idx, '!')

        # If the letter was already guessed, use a frequency-based fallback
        if guess_letter in self.guessed_letters:
            for letter, _ in self.full_dictionary_common_letter_sorted:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        return guess_letter


    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully start a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print(f"Guessing letter: {guess_letter}")

                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print(f"Sever response: {res}")
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print(f"Successfully finished game: {game_id}")
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print(f"Failed game: {game_id}. Because of: {reason}")
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exist or does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result


# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [5]:
api = HangmanAPI(access_token="310b8099fa3ecfca3fd74f409d9b61", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [19]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: e3a89d9c549a. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Guessing letter: !
Sever response: {'game_id': 'e3a89d9c549a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Guessing letter: e
Sever response: {'game_id': 'e3a89d9c549a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Guessing letter: i
Sever response: {'game_id': 'e3a89d9c549a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i _ _ _ _ i _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Guessing letter: a
Sever response: {'game_id': 'e3a89d9c549a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i a _ _ _ i _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Guessing letter: n
Sever response: {'game_id': 'e3a89d9c549a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i a _ _ n i _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Guessing 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    # api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: division by zero